# Recommending Without User Data: Content Filtering With NLP and Cosine Similarity

Using data previously cleaned, I'm going to use several methods of NLP along with cosine similarity to make a simple, initial movie recommendation system. This sytem can be used to recommend movies when users are brand new. There are several ways this can be used:

1. The user can search for similar movies to a movie they like when a user first signs up with the app or service, but movies will be very closely related (Iron Man, Iron Man II, Iron Man III, etc)
2. The app can use this feature to suggest similar movies for the User to initially rate. I.e if the user says they like Toy Story, movies with a high cosine similarity can be the next suggestions for a user to rate the movie. 
3. This method can be used to group similar movies together into recommended "pods". For example, "Japanese animation featuring strong female leads" might include Spirited Away and Princess Mononoke.


### Features Used For Recommendations

* **Rated** - The movie's rating
* **Year** - The Release Year
* **Genre** - The movie's genres
* **Writer** - Who wrote the screenplay
* **Actors** - Significant actors in the movie
* **Director** - who directed the movies
* **Key words** - Key words taken from the plot using [nltk_rake](https://pypi.org/project/rake-nltk/)

In [7]:
import pandas as pd
import numpy as np

import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import time 

# 1. Bag of Words and Count Vectorizer

In [75]:
movies = pd.read_csv('/Users/user/final-capstone/inprogress3.csv')
print(movies.shape)
movies.head()

/Users/user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(284303, 25)


,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Toy Story,1995,G,22-Nov-95,81 min,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A cowboy doll is profoundly threatened and jea...,...,95.0,8.3,"788,709",tt0114709,movie,20-Mar-01,NaN,Buena Vista,http://www.disney.com/ToyStory,True
1,Jumanji,1995,PG,15-Dec-95,104 min,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,...,39.0,7.0,"267,973",tt0113497,movie,25-Jan-00,NaN,Sony Pictures Home Entertainment,NaN,True
2,Grumpier Old Men,1995,PG-13,22-Dec-95,101 min,"Comedy, Romance",Howard Deutch,"Mark Steven Johnson (characters), Mark Steven ...","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",John and Max resolve to save their beloved bai...,...,46.0,6.6,"22,329",tt0113228,movie,18-Nov-97,NaN,Warner Home Video,NaN,True
3,Waiting to Exhale,1995,R,22-Dec-95,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...","Based on Terry McMillan's novel, this film fol...",...,NaN,5.8,"8,640",tt0114885,movie,6-Mar-01,NaN,Twentieth Century Fox Home Entertainment,NaN,True
4,Father of the Bride Part II,1995,PG,8-Dec-95,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...",George Banks must deal not only with the pregn...,...,49.0,6.0,"31,241",tt0113041,movie,9-May-00,NaN,Disney,NaN,True


In [76]:
movies = movies.drop(['website','response','type','ratings','metascore','dvd','box_office'],axis=1)


# Drop TV shows from the dataframe
ratings = ['TV-14', 'TV-PG', 'TV-MA', 'TV-G', 'TV-Y', 'TV-13', 'TV-Y7']

for rating in ratings:
    movies = movies[movies.rated != rating]
    
    
# Drop a few other misc TV shows that snuck through
titles = ['The Mahabharata', 'Centennial', 'Dominion Tank Police', 'Children of Dune', 'Helen of Troy']

for title in titles:
    movies = movies[movies['title'] != title]

# putting together the year with the movie title
movies['year'] = movies['year'].apply(lambda x: str(x))
movies['title_year'] = movies['title'].map(str) + ' (' + movies['year'] + ')'
    

movies['year'] = movies['year'].str.strip('–')
movies = movies[movies['year'] != 'nan']
movies['year'] = pd.to_numeric(movies['year'])
movies['year'] = movies.year.astype(int)

# Dropping movies that have fewer than 100 ratings on imdb only so that we can have computational power

movies['imdb_votes'] = movies['imdb_votes'].str.replace(",","")
movies['imdb_votes'] = pd.to_numeric(movies['imdb_votes'])
print(movies.shape)

(274600, 19)


In [77]:
vote_counts = movies[movies['imdb_votes'].notnull()]['imdb_votes'].astype('int')
vote_averages = movies[movies['imdb_rating'].notnull()]['imdb_rating'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['imdb_votes']+1 # added +1 - Dan
    R = x['imdb_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

movies['weighted_rating'] = movies.apply(weighted_rating, axis=1)

In [78]:
movies = movies.drop_duplicates(subset='title_year',keep='first')
movies.shape

(265417, 20)

In [81]:
movies = movies.dropna(subset = ['plot'])
movies.shape

(167469, 20)

In [82]:
# movies.to_csv('scraped-movies-with-wr-ty.csv', index=False)

In [33]:
movies.columns

Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'imdb_rating', 'imdb_votes', 'imdb_id', 'production', 'title_year',
       'weighted_rating'],
      dtype='object')

For computational reasons, we're going to pare down the data a bit. So far I've found that I can only scale the cosine similarity to about 51,000 movies. I do want to include popular movies, so I'm going to only include movies with over 100 IMDB votes and see how that does.

In [86]:
# Cosine Similarity Paring down - Want below 52,000 movies

threshold_votes = movies['imdb_votes'].quantile(.40)
quarter = movies['imdb_rating'].quantile(.30)

cos_movies = movies[(movies['imdb_votes'] >= threshold_votes) & (movies['imdb_rating'] >= quarter)
                    & (movies['year'] >= 1900) ]

cos_movies.shape

(54905, 20)

In [87]:
cos_movies = cos_movies[['title_year','weighted_rating','imdb_rating','imdb_id','rated','writer','genre','director','actors','plot','language','country', 'year']]

for column in cos_movies:
    cos_movies[column] = cos_movies[column].apply(lambda x: str(x))
    
# discarding the commas between the actors' full names
cos_movies['actors'] = cos_movies['actors'].map(lambda x: x.split(','))

# putting the genres in a list of words
cos_movies['genre'] = cos_movies['genre'].map(lambda x: x.lower().split(','))

cos_movies['director'] = cos_movies['director'].map(lambda x: x.split(','))


# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in cos_movies.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = [x.lower().replace(' ','') for x in row['director']]


# Getting rid of information between () in writers
cos_movies['writer'] = cos_movies['writer'].str.replace(r"\(.*\)","")

# discarding the commas between writer's names
cos_movies['writer'] = cos_movies['writer'].map(lambda x: x.split(','))

# putting the countries in a list of words
cos_movies['country'] = cos_movies['country'].map(lambda x: x.lower().split(','))

# putting the languages in a list of words
cos_movies['language'] = cos_movies['language'].map(lambda x: x.lower().split(','))

cos_movies['rated'] = cos_movies['rated'].str.replace("-","")

cos_movies['year'] = cos_movies['year'].map(lambda x: x.split(','))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in cos_movies.iterrows():
    row['writer'] = [x.lower().replace(' ','') for x in row['writer']]

cos_movies.head()

,title_year,weighted_rating,imdb_rating,imdb_id,rated,writer,genre,director,actors,plot,language,country,year
0,Toy Story (1995),8.29874801320629,8.3,tt0114709,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",A cowboy doll is profoundly threatened and jea...,[english],[usa],[1995]
1,Jumanji (1995),6.9981594078752325,7.0,tt0113497,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...",When two kids find and play a magical board ga...,"[english, french]",[usa],[1995]
2,Grumpier Old Men (1995),6.584943625757381,6.6,tt0113228,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...",John and Max resolve to save their beloved bai...,"[english, italian, german]",[usa],[1995]
3,Waiting to Exhale (1995),5.795795337651051,5.8,tt0114885,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...","Based on Terry McMillan's novel, this film fol...",[english],[usa],[1995]
4,Father of the Bride Part II (1995),5.99639712038929,6.0,tt0113041,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",George Banks must deal not only with the pregn...,[english],[usa],[1995]


In [88]:
cos_movies = cos_movies.drop(['weighted_rating','imdb_id','imdb_rating'],axis=1)
cos_movies.columns

Index(['title_year', 'rated', 'writer', 'genre', 'director', 'actors', 'plot',
       'language', 'country', 'year'],
      dtype='object')

In [89]:
# initializing the new column
cos_movies['Key_words'] = ""

for index, row in cos_movies.iterrows():
    plot = row['plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
cos_movies.drop(columns = ['plot'], inplace = True)

In [90]:
cos_movies.head()

,title_year,rated,writer,genre,director,actors,language,country,year,Key_words
0,Toy Story (1995),G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995],"[boy, room, top, toy, cowboy, doll, new, space..."
1,Jumanji (1995),PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995],"[finishing, man, trapped, play, magical, board..."
2,Grumpier Old Men (1995),PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995],"[save, beloved, bait, shop, turning, new, fema..."
3,Waiting to Exhale (1995),R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995],"[terry, mcmillan, film, follows, four, relatio..."
4,Father of the Bride Part II (1995),PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995],"[daughter, pregnancy, wife, unexpected, also, ..."


In [91]:
cos_movies.drop_duplicates(subset='title_year',keep='first')
cos_movies.shape

(54905, 10)

In [92]:
cos_movies.set_index('title_year', inplace = True)
cos_movies.head()

,rated,writer,genre,director,actors,language,country,year,Key_words
title_year,,,,,,,,,
Toy Story (1995),G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995],"[boy, room, top, toy, cowboy, doll, new, space..."
Jumanji (1995),PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995],"[finishing, man, trapped, play, magical, board..."
Grumpier Old Men (1995),PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995],"[save, beloved, bait, shop, turning, new, fema..."
Waiting to Exhale (1995),R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995],"[terry, mcmillan, film, follows, four, relatio..."
Father of the Bride Part II (1995),PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995],"[daughter, pregnancy, wife, unexpected, also, ..."


In [96]:
cos_movies['bag_of_words'] = ''
columns = cos_movies.columns
for index, row in cos_movies.iterrows():
    words = ''
    for col in columns:
        if col == 'rated':
            words = words.join(row[col]) + ' '
        elif col == 'imdb_rating':
            None
        else:
            words = words + ' '.join(row[col])+ ' '

    row['bag_of_words'] = words
    
cos_movies.drop(columns = [col for col in cos_movies.columns if col!= 'bag_of_words'], inplace = True)

In [97]:
# Filter out 'nan' from bag of words

cos_movies['bag_of_words'] = cos_movies['bag_of_words'].str.replace('nan','')


In [99]:
#cos_movies.to_csv('bagofwords-processed.csv')

In [104]:
cos_movies['bag_of_words'][0]

'G johnlasseter animation  adventure  comedy  family  fantasy johnlasseter tomhanks timallen donrickles jimvarney english usa 1995 boy room top toy cowboy doll new spaceman figure supplants profoundly threatened jealous  '

In [105]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(cos_movies['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(cos_movies.index)
indices[:5]

0                      Toy Story (1995)
1                        Jumanji (1995)
2               Grumpier Old Men (1995)
3              Waiting to Exhale (1995)
4    Father of the Bride Part II (1995)
Name: title_year, dtype: object

In [55]:
times = []

start = time.time()

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cos_sim_time = time.time() - start
times.append(cos_sim_time)
print("Runtime %0.2f" % (time.time() - start))
print(cosine_sim)

Runtime 390.00
[[1.         0.23732223 0.15694121 ... 0.03138824 0.07018624 0.03094922]
 [0.23732223 1.         0.12344268 ... 0.06172134 0.06900656 0.06085806]
 [0.15694121 0.12344268 1.         ... 0.11428571 0.03194383 0.05634362]
 ...
 [0.03138824 0.06172134 0.11428571 ... 1.         0.03194383 0.05634362]
 [0.07018624 0.06900656 0.03194383 ... 0.03194383 1.         0.03149704]
 [0.03094922 0.06085806 0.05634362 ... 0.05634362 0.03149704 1.        ]]


In [56]:
cosine_sim.shape

(57947, 57947)

In [57]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_15_indexes = list(score_series.iloc[1:16].index)
    
    # populating the list with the titles of the best 15 matching movies
    for i in top_15_indexes:
        recommended_movies.append(list(cos_movies.index)[i])
        
    return recommended_movies

In [58]:
def return_recs(title, orderby, df):
    title_a = recommendations(title)
    title_recs = df.loc[title_a].reset_index(inplace=False)
    title_recs = title_recs.set_index('title_year')
    #title_recs = df[df.index.isin(title_a)]
    print("Here are the movie recommendations for {}: ".format(title))
    if orderby == None:
        return title_recs[:10]
    else:
        return title_recs[:10].sort_values(by=[orderby], ascending=False)

In [9]:
movies = pd.read_csv('scraped-movies-with-wr-ty.csv')
#movies = movies.drop('Unnamed: 0', axis=1)
movies = movies.drop_duplicates(subset='title_year', keep='first')
movies = movies.set_index('title_year')

cols = ['rated','plot','year','genre','director','actors','language','weighted_rating']
movies = movies[cols]
movies.head()


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Toy Story (1995),G,A cowboy doll is profoundly threatened and jea...,1995,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",English,8.298748
Jumanji (1995),PG,When two kids find and play a magical board ga...,1995,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","English, French",6.998159
Grumpier Old Men (1995),PG-13,John and Max resolve to save their beloved bai...,1995,"Comedy, Romance",Howard Deutch,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...","English, Italian, German",6.584944
Waiting to Exhale (1995),R,"Based on Terry McMillan's novel, this film fol...",1995,"Comedy, Drama, Romance",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",English,5.795795
Father of the Bride Part II (1995),PG,George Banks must deal not only with the pregn...,1995,"Comedy, Family, Romance",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",English,5.996397


In [60]:
return_recs('Zootopia (2016)', 'weighted_rating', df = movies)

Here are the movie recommendations for Zootopia (2016): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
"Monsters, Inc. (2001)",G,"In order to power the city, monsters have to s...",2001,"Animation, Adventure, Comedy, Family, Fantasy","Pete Docter, David Silverman(co-director), Lee...","John Goodman, Billy Crystal, Mary Gibbs, Steve...",English,8.098755
Fantastic Mr. Fox (2009),PG,An urbane fox cannot resist returning to his f...,2009,"Animation, Adventure, Comedy, Crime, Drama, Fa...",Wes Anderson,"George Clooney, Meryl Streep, Jason Schwartzma...","English, French",7.895249
Who Framed Roger Rabbit (1988),PG,A toon-hating detective is a cartoon rabbit's ...,1988,"Animation, Adventure, Comedy, Crime, Family, F...",Robert Zemeckis,"Bob Hoskins, Christopher Lloyd, Joanna Cassidy...",English,7.695482
Tangled Ever After (2012),G,"Rapunzel and Flynn Rider have their wedding, b...",2012,"Animation, Short, Action, Comedy, Family, Musical","Nathan Greno, Byron Howard","Mandy Moore, Zachary Levi, Alan Dale, Paul F. ...",English,7.650855
Bolt (2008),PG,The canine star of a fictional sci-fi/action s...,2008,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams","John Travolta, Miley Cyrus, Susie Essman, Mark...",English,6.897374
Trolls (2016),PG,"After the Bergens invade Troll Village, Poppy,...",2016,"Animation, Adventure, Comedy, Family, Fantasy,...","Mike Mitchell, Walt Dohrn(co-director)","Anna Kendrick, Justin Timberlake, Zooey Descha...",English,6.495212
Hoodwinked! (2005),PG,"Little Red Riding Hood, the Wolf, the Woodsman...",2005,"Animation, Comedy, Crime, Family, Musical, Mys...","Cory Edwards, Todd Edwards(co-director), Tony ...","Anne Hathaway, Glenn Close, Jim Belushi, Patri...","English, Spanish",6.494235
Robots (2005),PG,"In a robot world, a young idealistic inventor ...",2005,"Animation, Adventure, Comedy, Family, Sci-Fi","Chris Wedge, Carlos Saldanha(co-director)","Paula Abdul, Halle Berry, Lucille Bliss, Terry...","English, Italian",6.298133
Planet 51 (2009),PG,An alien civilization is invaded by Astronaut ...,2009,"Animation, Adventure, Comedy, Family, Sci-Fi","Jorge Blanco, Javier Abad(co-director), Marcos...","Dwayne Johnson, Jessica Biel, Justin Long, Gar...","English, Spanish",6.096812


In [61]:
return_recs('The Lord of the Rings: The Fellowship of the Ring (2001)', 'weighted_rating', df = movies)

Here are the movie recommendations for The Lord of the Rings: The Fellowship of the Ring (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Lord of the Rings: The Return of the King (2003),PG-13,Gandalf and Aragorn lead the World of Men agai...,2003,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston","English, Quenya, Old English, Sindarin",8.899177
The Lord of the Rings: The Two Towers (2002),PG,While Frodo and Sam edge closer to Mordor with...,2002,"Adventure, Drama, Fantasy",Peter Jackson,"Bruce Allpress, Sean Astin, John Bach, Sala Baker","English, Sindarin, Old English",8.699152
Harry Potter and the Deathly Hallows: Part 2 (2011),PG-13,"Harry, Ron, and Hermione search for Voldemort'...",2011,"Adventure, Drama, Fantasy, Mystery",David Yates,"Ralph Fiennes, Michael Gambon, Alan Rickman, D...",English,8.098670
The Hobbit: An Unexpected Journey (2012),PG-13,"A reluctant Hobbit, Bilbo Baggins, sets out to...",2012,"Adventure, Family, Fantasy",Peter Jackson,"Ian McKellen, Martin Freeman, Richard Armitage...",English,7.898836
The Hobbit: The Desolation of Smaug (2013),PG-13,"The dwarves, along with Bilbo Baggins and Gand...",2013,"Adventure, Fantasy",Peter Jackson,"Ian McKellen, Martin Freeman, Richard Armitage...",English,7.798583
Thor (2011),PG-13,The powerful but arrogant god Thor (Chris Hems...,2011,"Action, Adventure, Fantasy, Sci-Fi",Kenneth Branagh,"Chris Hemsworth, Natalie Portman, Tom Hiddlest...",English,6.999272
What Dreams May Come (1998),PG-13,"Chris Nielsen dies in car crash one night, and...",1998,"Drama, Fantasy, Romance",Vincent Ward,"Robin Williams, Cuba Gooding Jr., Annabella Sc...",English,6.994813
The Lovely Bones (2009),PG-13,Centers on a young girl who has been murdered ...,2009,"Drama, Fantasy, Thriller",Peter Jackson,"Mark Wahlberg, Rachel Weisz, Susan Sarandon, S...",English,6.697392
The Long and Short of It (2003),NaN,A workman gets some unexpected help from two p...,2003,"Short, Drama",Sean Astin,"Andrew Lesnie, Praphaphorn 'Fon' Chansantor, P...",English,6.627800


In [62]:
recommendations('Spirited Away (2001)')

['Ponyo (2008)',
 'My Neighbor Totoro (1988)',
 "Howl's Moving Castle (2004)",
 'Porco Rosso (1992)',
 'The Cat Returns (2002)',
 "Kiki's Delivery Service (1989)",
 'Castle in the Sky (1986)',
 'Nausicaä of the Valley of the Wind (1984)',
 'Nutcracker Fantasy (1979)',
 "Puss 'N Boots Travels Around the World (1976)",
 'My Neighbors the Yamadas (1999)',
 "Harry Potter and the Sorcerer's Stone (2001)",
 'Mei and the Kitten Bus (2002)',
 'Onigamiden - Legend of the Millennium Dragon (2011)',
 'Pom Poko (1994)']

In [63]:
return_recs('Spirited Away (2001)', 'weighted_rating', df = movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Howl's Moving Castle (2004),PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",Japanese,8.196626
My Neighbor Totoro (1988),G,When two girls move to the country to be near ...,1988,"Animation, Family, Fantasy",Hayao Miyazaki,"Noriko Hidaka, Chika Sakamoto, Shigesato Itoi,...",Japanese,8.196011
Castle in the Sky (1986),PG,A young boy and a girl with a magic crystal mu...,1986,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,"Anna Paquin, James Van Der Beek, Cloris Leachm...",Japanese,8.092852
Porco Rosso (1992),PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...",Japanese,7.786986
Ponyo (2008),G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",Japanese,7.692918
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Mei and the Kitten Bus (2002),NaN,This is the story of Mei and her friend the ki...,2002,"Animation, Short, Comedy",Hayao Miyazaki,"Hayao Miyazaki, Chika Sakamoto",Japanese,7.492856
Pom Poko (1994),PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...","Japanese, English",7.367225
The Cat Returns (2002),G,"After helping a cat, a 17-year-old girl finds ...",2002,"Animation, Adventure, Comedy, Family, Fantasy,...",Hiroyuki Morita,"Chizuru Ikewaki, Yoshihiko Hakamada, Aki Maeda...","Japanese, English",7.284617


In [64]:
return_recs('Titanic (1997)', 'weighted_rating', df = movies)

Here are the movie recommendations for Titanic (1997): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Notebook (2004),PG-13,A poor yet passionate young man falls in love ...,2004,"Drama, Romance",Nick Cassavetes,"Tim Ivey, Gena Rowlands, Starletta DuPois, Jam...",English,7.898228
500 Days of Summer (2009),PG-13,An offbeat romantic comedy about a woman who d...,2009,"Comedy, Drama, Romance",Marc Webb,"Joseph Gordon-Levitt, Zooey Deschanel, Geoffre...","English, French, Swedish",7.698267
Ghosts of the Abyss (2003),G,"James Cameron and Bill Paxton, director and ac...",2003,"Documentary, Family",James Cameron,"Bill Paxton, John Broadwater, Lori Johnston, C...",English,6.810055
The Luzhin Defence (2000),PG-13,Two worlds collide when an eccentric genius fa...,2000,"Drama, Romance",Marleen Gorris,"John Turturro, Emily Watson, Geraldine James, ...","English, Italian",6.796942
Where the Heart Is (2000),PG-13,A pregnant 17-year-old rebuilds her life after...,2000,"Comedy, Drama, Romance",Matt Williams,"Natalie Portman, Ashley Judd, Stockard Channin...",English,6.786285
Crazy/Beautiful (2001),PG-13,"At Pacific Palisades High, a poor Latino falls...",2001,"Drama, Romance",John Stockwell,"Kirsten Dunst, Jay Hernandez, Bruce Davison, H...",English,6.486972
Dear John (2010),PG-13,A romantic drama about a soldier who falls for...,2010,"Drama, Romance, War",Lasse Hallström,"Channing Tatum, Amanda Seyfried, Richard Jenki...",English,6.298220
Stanley & Iris (1990),PG-13,A struggling widow falls in love with an illit...,1990,"Drama, Romance",Martin Ritt,"Jane Fonda, Robert De Niro, Swoosie Kurtz, Mar...",English,6.266530
Tadpole (2002),PG-13,Coming-of-age story about a suave 15-year-old ...,2002,"Comedy, Drama, Romance",Gary Winick,"Aaron Stanford, Kate Mara, Robert Iler, Peter ...","English, French",6.167841


In [65]:
return_recs('Wings of Desire (1987)', 'weighted_rating', movies)

Here are the movie recommendations for Wings of Desire (1987): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Alice in the Cities (1974),Not Rated,A German journalist is saddled with a nine-yea...,1974,Drama,Wim Wenders,"Rüdiger Vogler, Yella Rottländer, Lisa Kreuzer...","German, English, Dutch",7.901960
"Lacombe, Lucien (1974)",R,"In 1944, an 18-year old boy from small-town Fr...",1974,"Drama, Romance, War",Louis Malle,"Pierre Blaise, Aurore Clément, Holger Löwenadl...","French, German, English",7.667056
Pina (2011),PG,"A tribute to the late German choreographer, Pi...",2011,"Documentary, Music",Wim Wenders,"Regina Advento, Malou Airaudo, Ruth Amarante, ...","German, French, English, Spanish, Croatian, It...",7.644937
The American Friend (1977),Not Rated,"Tom Ripley, who deals in forged art, suggests ...",1977,"Crime, Drama, Mystery, Thriller",Wim Wenders,"Dennis Hopper, Bruno Ganz, Lisa Kreuzer, Gérar...","German, English, French",7.346569
"Faraway, So Close! (1993)",PG-13,A group of angels in the German capital look l...,1993,"Drama, Fantasy, Romance",Wim Wenders,"Otto Sander, Peter Falk, Horst Buchholz, Mikha...","German, French, English, Italian, Russian",7.214520
Silent Light (2007),NaN,"In a Mennonite community in Mexico, a father's...",2007,"Drama, Romance",Carlos Reygadas,"Cornelio Wall, Miriam Toews, Maria Pankratz, P...","Low German, Spanish, French, English",7.190581
Tokyo-Ga (1985),NaN,"Moved by the work of director Yasujirô Ozu, Wi...",1985,Documentary,Wim Wenders,"Chishû Ryû, Werner Herzog, Yûharu Atsuta","English, Japanese, German",7.143067
Voyager (1991),PG-13,Walter Faber has survived a crash with an airp...,1991,"Drama, Romance",Volker Schlöndorff,"Sam Shepard, Julie Delpy, Barbara Sukowa, Diet...","German, English, Spanish, French, Greek",6.577521
Land of Plenty (2004),NOT RATED,A drama that investigates anxiety and disillus...,2004,"Drama, War",Wim Wenders,"Michelle Williams, John Diehl, Shaun Toub, Wen...",English,6.422802


In [66]:
return_recs('Amélie (2001)', 'weighted_rating', movies)

Here are the movie recommendations for Amélie (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Stolen Kisses (1968),R,"After being discharged from the army, Antoine ...",1968,"Comedy, Drama, Romance",François Truffaut,"Jean-Pierre Léaud, Delphine Seyrig, Claude Jad...","French, English",7.725378
Delicatessen (1991),R,Post-apocalyptic surrealist black comedy about...,1991,"Comedy, Crime","Marc Caro, Jean-Pierre Jeunet","Pascal Benezech, Dominique Pinon, Marie-Laure ...",French,7.689895
A Little Romance (1979),PG,A French boy (Daniel) and an American girl (La...,1979,"Adventure, Comedy, Romance",George Roy Hill,"Laurence Olivier, Diane Lane, Thelonious Berna...","English, French, Italian",7.366602
Rendezvous in Paris (1995),NaN,Three stories of love and coincidence around t...,1995,"Comedy, Romance",Éric Rohmer,"Clara Bellar, Antoine Basler, Mathias Mégard, ...",French,7.114253
Romantics Anonymous (2010),PG-13,What happens when a man and a woman share a co...,2010,"Comedy, Romance",Jean-Pierre Améris,"Benoît Poelvoorde, Isabelle Carré, Lorella Cra...",French,6.855191
Brief Crossing (2001),Unrated,A young French man and an older English woman ...,2001,"Comedy, Drama, Romance",Catherine Breillat,"Sarah Pratt, Gilles Guillain, Marc Filipi, Laë...","French, English",6.653817
27 Missing Kisses (2000),NaN,"The summerly adventures of teen Sybilla, who f...",2000,"Comedy, Drama, Romance",Nana Dzhordzhadze,"Nutsa Kukhianidze, Evgeniy Sidikhin, Shalva Ia...","Georgian, Russian, French, English",6.547627
Beautiful Lies (2010),NaN,A hairdresser forwards a passionate love lette...,2010,"Comedy, Romance",Pierre Salvadori,"Audrey Tautou, Nathalie Baye, Sami Bouajila, S...","French, Chinese, English",6.456519
Mr. Bean's Holiday (2007),G,Mr. Bean wins a trip to Cannes where he unwitt...,2007,"Comedy, Family",Steve Bendelack,"Rowan Atkinson, Steve Pemberton, Lily Atkinson...","English, French, Spanish, Russian",6.397489


In [67]:
return_recs("Baby Driver (2017)", 'weighted_rating', movies)

Here are the movie recommendations for Baby Driver (2017): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Mr. Holland's Opus (1995),PG,A frustrated composer finds fulfillment as a h...,1995,"Drama, Music",Stephen Herek,"Richard Dreyfuss, Glenne Headly, Jay Thomas, O...","English, American Sign Language",7.281932
Mr. Majestyk (1974),PG,A melon farmer battles organized crime and a h...,1974,"Action, Crime, Drama, Romance, Thriller",Richard Fleischer,"Charles Bronson, Al Lettieri, Linda Cristal, L...",English,6.741095
Absolute Power (1997),R,A career thief witnesses a horrific crime invo...,1997,"Action, Crime, Drama, Thriller",Clint Eastwood,"Clint Eastwood, Gene Hackman, Ed Harris, Laura...","English, Spanish",6.691833
New Jack City (1991),R,A crime lord ascends to power and becomes mega...,1991,"Action, Crime, Drama, Thriller",Mario Van Peebles,"Wesley Snipes, Ice-T, Allen Payne, Chris Rock",English,6.588204
Heist (2001),R,A career jewel thief finds himself at tense od...,2001,"Action, Crime, Drama, Thriller",David Mamet,"Gene Hackman, Danny DeVito, Delroy Lindo, Sam ...",English,6.490304
Q & A (1990),R,A young district attorney seeking to prove a c...,1990,"Crime, Drama, Thriller",Sidney Lumet,"Nick Nolte, Timothy Hutton, Armand Assante, Pa...","English, Spanish",6.440881
The Principal (1987),R,A teacher is assigned to be the principal of a...,1987,"Action, Crime, Drama, Thriller",Christopher Cain,"Jim Belushi, Louis Gossett Jr., Rae Dawn Chong...",English,6.273172
Wrong Turn at Tahoe (2009),R,A small-time crime boss kills a drug dealer wi...,2009,"Action, Crime, Drama, Thriller",Franck Khalfoun,"Cuba Gooding Jr., Miguel Ferrer, Harvey Keitel...",English,6.062140
The Take (2007),R,"After he's shot during a heist in East L.A., a...",2007,"Action, Crime, Drama",Brad Furman,"John Leguizamo, Tyrese Gibson, Rosie Perez, Bo...",English,5.784619


In [68]:
return_recs("Spirited Away (2001)", 'weighted_rating', movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Howl's Moving Castle (2004),PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",Japanese,8.196626
My Neighbor Totoro (1988),G,When two girls move to the country to be near ...,1988,"Animation, Family, Fantasy",Hayao Miyazaki,"Noriko Hidaka, Chika Sakamoto, Shigesato Itoi,...",Japanese,8.196011
Castle in the Sky (1986),PG,A young boy and a girl with a magic crystal mu...,1986,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,"Anna Paquin, James Van Der Beek, Cloris Leachm...",Japanese,8.092852
Porco Rosso (1992),PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...",Japanese,7.786986
Ponyo (2008),G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",Japanese,7.692918
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Mei and the Kitten Bus (2002),NaN,This is the story of Mei and her friend the ki...,2002,"Animation, Short, Comedy",Hayao Miyazaki,"Hayao Miyazaki, Chika Sakamoto",Japanese,7.492856
Pom Poko (1994),PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...","Japanese, English",7.367225
The Cat Returns (2002),G,"After helping a cat, a 17-year-old girl finds ...",2002,"Animation, Adventure, Comedy, Family, Fantasy,...",Hiroyuki Morita,"Chizuru Ikewaki, Yoshihiko Hakamada, Aki Maeda...","Japanese, English",7.284617


In [69]:
return_recs("Alice in Wonderland (2010)", 'weighted_rating', movies)

Here are the movie recommendations for Alice in Wonderland (2010): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Spirited Away (2001),PG,"During her family's move to the suburbs, a sul...",2001,"Animation, Adventure, Family, Fantasy, Mystery","Hayao Miyazaki, Kirk Wise","Rumi Hiiragi, Miyu Irino, Mari Natsuki, Takash...","Japanese, English",8.597999
How to Train Your Dragon (2010),PG,A hapless young Viking who aspires to hunt dra...,2010,"Animation, Action, Adventure, Family, Fantasy","Dean DeBlois, Chris Sanders","Jay Baruchel, Gerard Butler, Craig Ferguson, A...",English,8.098501
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Bridge to Terabithia (2007),PG,A preteen's life turns upside down when he bef...,2007,"Adventure, Drama, Family, Fantasy",Gabor Csupo,"Josh Hutcherson, AnnaSophia Robb, Zooey Descha...",English,7.195655
Legend of the Guardians: The Owls of Ga'Hoole (2010),PG,When a young owl is abducted by an evil Owl ar...,2010,"Animation, Action, Adventure, Family, Fantasy",Zack Snyder,"Emily Barclay, Abbie Cornish, Essie Davis, Adr...",English,6.993311
Mirrormask (2005),PG,"In a fantasy world of opposing kingdoms, a fif...",2005,"Adventure, Fantasy",Dave McKean,"Stephanie Leonidas, Jason Barry, Rob Brydon, G...",English,6.879333
Charlie and the Chocolate Factory (2005),PG,A young boy wins a tour through the most magni...,2005,"Adventure, Comedy, Family, Fantasy, Musical",Tim Burton,"Johnny Depp, Freddie Highmore, David Kelly, He...",English,6.599121
Zathura: A Space Adventure (2005),PG,Two young brothers are drawn into an intergala...,2005,"Action, Adventure, Comedy, Family, Fantasy, Sc...",Jon Favreau,"Jonah Bobo, Josh Hutcherson, Dax Shepard, Kris...",English,6.197735
Balto: Wolf Quest (2002),G,Balto and his daughter Aleu embark on a journe...,2002,"Animation, Adventure, Family, Fantasy",Phil Weinstein,"Jodi Benson, David Carradine, Lacey Chabert, M...",English,6.154322


In [70]:
return_recs("Seven Samurai (1954)", 'weighted_rating', movies)

Here are the movie recommendations for Seven Samurai (1954): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Rashomon (1950),Not Rated,The rape of a bride and the murder of her samu...,1950,"Crime, Drama, Mystery",Akira Kurosawa,"Toshirô Mifune, Machiko Kyô, Masayuki Mori, Ta...",Japanese,8.292579
Yojimbo (1961),Not Rated,A crafty ronin comes to a town divided by two ...,1961,"Action, Drama, Thriller",Akira Kurosawa,"Toshirô Mifune, Tatsuya Nakadai, Yôko Tsukasa,...",Japanese,8.289807
Ikiru (1952),Not Rated,A bureaucrat tries to find a meaning in his li...,1952,Drama,Akira Kurosawa,"Takashi Shimura, Shin'ichi Himori, Haruo Tanak...",Japanese,8.283107
Red Beard (1965),Not Rated,"In 19th century Japan, a rough tempered yet ch...",1965,Drama,Akira Kurosawa,"Toshirô Mifune, Yûzô Kayama, Tsutomu Yamazaki,...",Japanese,8.231848
Throne of Blood (1957),Not Rated,"A war-hardened general, egged on by his ambiti...",1957,Drama,Akira Kurosawa,"Toshirô Mifune, Isuzu Yamada, Takashi Shimura,...",Japanese,8.077127
The Hidden Fortress (1958),Not Rated,"Lured by gold, two greedy peasants escort a ma...",1958,"Adventure, Drama",Akira Kurosawa,"Toshirô Mifune, Minoru Chiaki, Kamatari Fujiwa...",Japanese,8.069465
Sanjuro (1962),Not Rated,A crafty samurai helps a young man and his fel...,1962,"Action, Crime, Drama, Thriller",Akira Kurosawa,"Toshirô Mifune, Tatsuya Nakadai, Keiju Kobayas...",Japanese,8.067454
Samurai I: Musashi Miyamoto (1954),NOT RATED,Depicts the early life of the legendary warrio...,1954,"Action, Adventure, Biography, Drama, History, ...",Hiroshi Inagaki,"Toshirô Mifune, Rentarô Mikuni, Kurôemon Onoe,...",Japanese,7.403536
The Lower Depths (1957),Not Rated,"In a Japanese slum, various residents play out...",1957,Drama,Akira Kurosawa,"Toshirô Mifune, Isuzu Yamada, Kyôko Kagawa, Ga...",Japanese,7.351771


In [71]:
return_recs("Schindler's List (1993)", 'weighted_rating', movies)

Here are the movie recommendations for Schindler's List (1993): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Pianist (2002),R,A Polish Jewish musician struggles to survive ...,2002,"Biography, Drama, Music, War",Roman Polanski,"Adrien Brody, Emilia Fox, Michal Zebrowski, Ed...","English, German, Russian",8.498336
The Great Escape (1963),Not Rated,Allied prisoners of war plan for several hundr...,1963,"Adventure, Drama, History, Thriller, War",John Sturges,"Steve McQueen, James Garner, Richard Attenboro...","English, German, French, Russian, Spanish",8.195362
The Imitation Game (2014),PG-13,"During World War II, the English mathematical ...",2014,"Biography, Drama, Thriller, War",Morten Tyldum,"Benedict Cumberbatch, Keira Knightley, Matthew...","English, German",7.998578
Nuremberg (2000–),NaN,The dramatized account of the war crime trials...,2000,"Drama, History, War",NaN,"Alec Baldwin, Brian Cox, Christopher Plummer","English, German",7.283721
Hitler: The Rise of Evil (2003),NaN,"A unique slant, profiling the life of Adolf Hi...",2003,"Biography, Drama, History, War",NaN,"Peter Stormare, Friedrich von Thun, Peter O'To...",English,7.269667
Labyrinth of Lies (2014),R,A story that exposes the conspiracy of promine...,2014,"Drama, History",Giulio Ricciarelli,"Alexander Fehling, André Szymanski, Friederike...","German, English",7.243343
In Darkness (2011),R,A dramatization of one man's rescue of Jewish ...,2011,"Drama, War",Agnieszka Holland,"Robert Wieckiewicz, Benno Fürmann, Agnieszka G...","Polish, Ukrainian, Yiddish, German",7.237262
Valkyrie (2008),PG-13,"A dramatization of the July 20, 1944 assassina...",2008,"Drama, History, Thriller, War",Bryan Singer,"Tom Cruise, Kenneth Branagh, Bill Nighy, Tom W...","English, German",7.097501
The Eagle Has Landed (1976),PG,A German plot to kidnap Sir Winston Churchill ...,1976,"Adventure, Drama, Thriller, War",John Sturges,"Michael Caine, Donald Sutherland, Robert Duval...","English, German, Polish",6.873865


In [72]:
return_recs("Inception (2010)", 'weighted_rating', movies)

Here are the movie recommendations for Inception (2010): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Interstellar (2014),PG-13,A team of explorers travel through a wormhole ...,2014,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Ellen Burstyn, Matthew McConaughey, Mackenzie ...",English,8.599145
The Prestige (2006),PG-13,"After a tragic accident, two stage magicians e...",2006,"Drama, Mystery, Sci-Fi, Thriller",Christopher Nolan,"Hugh Jackman, Christian Bale, Michael Caine, P...",English,8.498997
The Dark Knight Rises (2012),PG-13,Eight years after the Joker's reign of anarchy...,2012,"Action, Thriller",Christopher Nolan,"Christian Bale, Gary Oldman, Tom Hardy, Joseph...","English, Arabic",8.399257
Following (1998),R,A young writer who follows strangers for mater...,1998,"Crime, Mystery, Thriller",Christopher Nolan,"Jeremy Theobald, Alex Haw, Lucy Russell, John ...",English,7.590774
Transformers: Dark of the Moon (2011),PG-13,The Autobots learn of a Cybertronian spacecraf...,2011,"Action, Adventure, Sci-Fi",Michael Bay,"Shia LaBeouf, Rosie Huntington-Whiteley, Josh ...",English,6.299377
Next (2007),PG-13,A Las Vegas magician who can see into the futu...,2007,"Action, Adventure, Crime, Fantasy, Sci-Fi, Thr...",Lee Tamahori,"Nicolas Cage, Julianne Moore, Jessica Biel, Th...","English, French, German, Serbian",6.198644
Reign of Fire (2002),PG-13,A brood of fire-breathing dragons emerges from...,2002,"Action, Adventure, Fantasy, Sci-Fi, Thriller",Rob Bowman,"Christian Bale, Matthew McConaughey, Izabella ...",English,6.198413
Jumper (2008),PG-13,A teenager with teleportation abilities sudden...,2008,"Action, Adventure, Sci-Fi, Thriller",Doug Liman,"Hayden Christensen, Jamie Bell, Rachel Bilson,...","English, Italian, Japanese, Mandarin",6.099441
Battleship (2012),PG-13,A fleet of ships is forced to do battle with a...,2012,"Action, Adventure, Sci-Fi, Thriller",Peter Berg,"Taylor Kitsch, Alexander Skarsgård, Rihanna, B...","English, Japanese",5.799831


In [73]:
return_recs("Iron Man (2008)", 'weighted_rating', movies)

Here are the movie recommendations for Iron Man (2008): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Avengers (2012),PG-13,Earth's mightiest heroes must come together an...,2012,"Action, Adventure, Sci-Fi",Joss Whedon,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...","English, Russian, Hindi",8.099227
Spider-Man (2002),PG-13,"When bitten by a genetically modified spider, ...",2002,"Action, Adventure, Sci-Fi",Sam Raimi,"Tobey Maguire, Willem Dafoe, Kirsten Dunst, Ja...",English,7.299045
Iron Man 3 (2013),PG-13,When Tony Stark's world is torn apart by a for...,2013,"Action, Adventure, Sci-Fi",Shane Black,"Robert Downey Jr., Gwyneth Paltrow, Don Cheadl...",English,7.199172
Transformers (2007),PG-13,An ancient struggle between two Cybertronian r...,2007,"Action, Adventure, Sci-Fi",Michael Bay,"Shia LaBeouf, Megan Fox, Josh Duhamel, Tyrese ...","English, Spanish",7.099063
Iron Man 2 (2010),PG-13,With the world now aware of his identity as Ir...,2010,"Action, Adventure, Sci-Fi",Jon Favreau,"Robert Downey Jr., Gwyneth Paltrow, Don Cheadl...","English, French, Russian",6.999241
Justice League: The New Frontier (2008),PG-13,"In the 1950s, a new generation of superheroes ...",2008,"Animation, Action, Adventure, Fantasy, Sci-Fi",Dave Bullock,"David Boreanaz, Miguel Ferrer, Neil Patrick Ha...",English,6.970688
The Incredible Hulk (2008),PG-13,"Bruce Banner, a scientist on the run from the ...",2008,"Action, Adventure, Sci-Fi",Louis Leterrier,"Edward Norton, Liv Tyler, Tim Roth, William Hurt","English, Portuguese, Spanish",6.699048
Dune (1984),PG-13,A Duke's son leads desert warriors against the...,1984,"Action, Adventure, Sci-Fi",David Lynch,"Francesca Annis, Leonardo Cimino, Brad Dourif,...",English,6.596938
Aftermath: Population Zero (2008),NaN,Ever wondered what the world would be like wit...,2008,"Documentary, Sci-Fi",NaN,Reg E. Cathey,English,6.547931


# Key Findings

1. This is a good initial filter for a website or app that gives movie recommendations for movies that are similar
2. Depending on the movie, key metrics like diveristy/variety differ. Below is analysis for a few of the recommendations:

## 1. The Lord of the Rings: The Fellowship of the Ring
All the movies that are also in the Lord of the Rings Franchise are returned, along with Harry Potter. But it also suggests a variety of other movies that are somewhat similar in tone, though none of them seem to be High Fantasy the same way that Lord of the Rings is. It's important to note, however, that there are two filters: The first filters for cosine similarity, the second filters the results by the weighted IMDB rating. By cosine similarity **only**, we also get movies from the Hobbit franchise, which are closely related to Lord of the Rings as it exists in the same Universe. However, these movies had a lower weighted rating on IMDB than other movies, so were suggested later. 

Movies recommended **ordered by cosine similarity**:

1. **'The Lord of the Rings: The Return of the King'**, also in the franchise
2. **'The Lord of the Rings: The Two Towers'**, also in the franchise
3. **'The Lord of the Rings (1978)**, the 1978 animated film of the same movie
4. **'Harry Potter and the Deathly Hallows: Part 2'** a similar movie in terms of high fantasy and plot
5. **'The Hobbit: The Desolation of Smaug'** also in the franchise
6. **'The Hobbit: An Unexpected Journey'** also in the franchise
7. **'The Long and Short of It'** - a short film made by crew during the filming of LoTR: The Two Towers
8. **'What Dreams May Come'** - a 1998 film that features tangential similarities, a 'journey' through a foreign land as the main character traverses through the afterlife in various forms, I'm not sure if this would be popular for lovers of LoTR as it's a different kind of 'fantasy' and 'drama'
9. **'The Hobbit: The Battle of the Five Armies'** another Hobbit film
10. **'The Lovely Bones'** - Another movie that involves the afterlife, I'm not sure this would be popular among people who love LoTR
11. **'The Price of Milk'** - A romantic comedy filmed in New Zealand, where the LoTR franchise was filmed
12. **'Hunt for the Wilderpeople'** - Another example of I think the algorithm using 'drama'
13. **'Thor'** - probably more of a movie that would be enjoyed by LoTR fans, as it's fantasy and deals more with 'war'
14. **'Captain Marvel'** - Similar to Thor, I think this is closer to what users would want to be recommended
15. **'Camille'** - has to do with journey through the afterlife


### Overall takeaway for LoTR: Fellowship Recommendations

The recommendations all have to do with a hero's journey, loss, and in many cases the afterlife. Most of them have some sort of fantastical element, though the recommendations fall short of what LoTR fans might like. I don't think this is a great filter to show to people who love recommendations, but think it could be improved if there were some more keywords (perhaps user-inputted keywords like 'High fantasy' and 'war'). 

The MovieLens recommender also recommends Star Wars and Indiana Jones based on Lord of the Rings. For computational time, this version of the filter only uses movies from 1988 and beyond, and therefore misses Star Wars and Indiana Jones, for the most part, but not the new Star Wars movies, which I suppose are not close enough to be recommended.

It's important to note that, in terms of themes, these **are good picks**. These movies all have thematic links, about loss and sacrifice and the inevitability of entropy and how power attracts the corruptible. Except Joe Dirt, I'm not sure what that's about.



## 2. Spirited Away
The results for *Spirited Away*, the award-winning 2001 animation, are a little more consistent across the board. Most of the other movies are also Japanese animation from the same studio, Studio Ghibli, making the diversity and variety of this set lower than Lord of the Rings, which recommended quite a few different genres, directors, and film ideas (which is good and bad, in its own way).

Movies recommended **filtered by cosine similarity and ordered by the weighted IMDB rating**:

* **Howl's Moving Castle** - deals with many similar themes and is from the same studio/director
* **My Neighbor Totoro** - also from Miyazaki and Studio Ghibli, this movie also features strong female leads, as two girls who have moved to a new home (similar) encounter strange creatures and beasts
* **Castle in the Sky** - Another Miyazaki with similar qualities, female leads, strange journies and beasts (to be fair, these themes run through most of Miyazaki's works)
* **Porco Rosso** - Also from Miyazaki
* **Ponyo** - Also from Miyazaki
* **Harry Potter and the Sorcerer's Stone** - a similar theme and idea of a child taken away from their parents and treated poorly by those around them (Harry loses his parents, Chihiro in Spirited Away has her parents turned into pigs). Other, magical themes run through both of these stories, making this a decent recommendation.
* **Mei and the Kitten Bus** - a spinoff short film with characters from Miyazaki's My Neighbor Totoro, this takes the younger girl Mei and the 'kitten bus' (based off the cat bus) from that film.
* **Pom Poko** - from Studio Ghibli / Miyazaki
* **The Cat Returns** - from Studio Ghibli / Miyazaki
* **My Neighbors the Yamadas** - from Studio Ghibli / Miyazaki


### Overall takeway for Spirited Away:
These recommendations are certainly more consistent and better than they were for Lord of Rings, but are a little strict in terms of who the director is, for example. I would have loved to get back some other Japanese animation (however I don't even know if that's in my dataframe). However, again, the algorithm has returned movies that rest on pretty similar thematic lines.

## Overall Findings

The algorithm is good at finding underlying thematic tones of movies and returning them, even if they aren't strictly in the same genre. Some tuning to the algorithm would perhaps enhance it, playing with NLP to see what could be done with n-grams instead of just a bag of words, or other processes like word2vec, as well as lemmatization or stemming, which I did not do in this iteration of the filter. Overall, I'm happy with how it performs, and thin it would be a good initial interface for someone to interact with.

## Ideas for further tuning and research

I'd love to try this out with more tags. For example, *Spirited Away*'s original language was Japanese, but it was translated and widely released into English, and Studio Ghibli is partnered with Pixar, who is owned by Disney. These lines could help bridge the gaps between certain movies. In an earlier version of this filter, Spirited Away returned *Alice in Wonderland*, which thrilled me. That doesn't seem to be the case this time, which is disappointing, but makes me wonder about how we might get a machine to see those gaps and leap across them, especially using a distance function such as cosine similarity. 

# Part 2. Using TF-IDF Vectorizer and Dot Product

In [100]:
tfidf_movs = pd.read_csv('bagofwords-processed.csv')

In [101]:
tfidf_movs.head()

,title_year,bag_of_words
0,Toy Story (1995),G johnlasseter animation adventure comedy f...
1,Jumanji (1995),PG jonathanhensleigh adventure comedy family...
2,Grumpier Old Men (1995),PG13 markstevenjohnson markstevenjohnson comed...
3,Waiting to Exhale (1995),R terrymcmillan comedy drama romance forestw...
4,Father of the Bride Part II (1995),PG alberthackett comedy family romance charl...


In [102]:
tfidf_movs.shape

(54905, 2)

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), stop_words='english')
tfidf_matrix = vectorizer.fit_transform(tfidf_movs['bag_of_words'])

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Since I have used the TF-IDF Vectorizer, calculating the Dot Product will directly give me the Cosine Similarity Score. Therefore, I will use sklearn’s linear_kernel instead of cosine_similarities since it is much faster.

In [7]:
from sklearn.metrics.pairwise import linear_kernel


start = time.time()

tf_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
tf_sim_time = time.time() - start
#times.append(tf_sim_time)
print("Runtime %0.2f" % (time.time() - start))

Runtime 373.43


In [8]:
tf_cosine_sim

array([[1.00000000e+00, 5.05979397e-02, 2.22538670e-02, ...,
        6.40892479e-04, 1.02767020e-02, 1.05732619e-03],
       [5.05979397e-02, 1.00000000e+00, 1.87064035e-02, ...,
        6.35664257e-04, 1.01928676e-02, 3.60122050e-03],
       [2.22538670e-02, 1.87064035e-02, 1.00000000e+00, ...,
        7.87206107e-03, 1.06307053e-03, 2.49502846e-03],
       ...,
       [6.40892479e-04, 6.35664257e-04, 7.87206107e-03, ...,
        1.00000000e+00, 6.19474500e-03, 4.33834720e-03],
       [1.02767020e-02, 1.01928676e-02, 1.06307053e-03, ...,
        6.19474500e-03, 1.00000000e+00, 1.00102538e-03],
       [1.05732619e-03, 3.60122050e-03, 2.49502846e-03, ...,
        4.33834720e-03, 1.00102538e-03, 1.00000000e+00]])

In [44]:
# Build a 1-dimensional array with movie titles
titles = tfidf_movs['title_year']
indices = pd.Series(tfidf_movs.index, index=tfidf_movs['title_year'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def tfidf_recommendations(title):
    
    '''
    This function returns a list of movies with the highest similarity scores other than itself
    '''
    
    idx = indices[title]
    sim_scores = list(enumerate(tf_cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


def return_tfidf_rec(title, orderby, df):
    
    '''
    using the tfidf_recommendations function, filters and sorts the recommendations by a column. Can order results
    by weighted rating or any other column value, if None then results are returned in order by cosine similarity
    '''
    
    title_a = tfidf_recommendations(title).tolist()
    title_recs = df.loc[title_a].reset_index(inplace=False)
    title_recs = title_recs.set_index('title_year')
    print("Here are the movie recommendations for {}: ".format(title))
    if orderby == None:
        return title_recs[:10]
    else:
        return title_recs[:10].sort_values(by=[orderby], ascending=False)

In [72]:
movies = pd.read_csv('/Users/user/final-capstone/scraped-movies-with-wr-ty.csv')

cols = ['title_year','rated','genre','director','plot','language','weighted_rating']
movies = movies[cols]
movies = movies.drop_duplicates(subset='title_year',keep='first')
movies = movies.set_index('title_year')

In [45]:
return_tfidf_rec('Spirited Away (2001)', None, movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Nausicaä of the Valley of the Wind (1984),PG,"Animation, Adventure, Fantasy, Sci-Fi",Hayao Miyazaki,Warrior and pacifist Princess Nausicaä despera...,"Japanese, English",8.092846
Porco Rosso (1992),PG,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"In 1930s Italy, a veteran World War I pilot is...",Japanese,7.786986
Castle in the Sky (1986),PG,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,A young boy and a girl with a magic crystal mu...,Japanese,8.092852
Mei and the Kitten Bus (2002),NaN,"Animation, Short, Comedy",Hayao Miyazaki,This is the story of Mei and her friend the ki...,Japanese,7.492856
Howl's Moving Castle (2004),PG,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,When an unconfident young woman is cursed with...,Japanese,8.196626
Ponyo (2008),G,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,A five-year-old boy develops a relationship wi...,Japanese,7.692918
My Neighbor Totoro (1988),G,"Animation, Family, Fantasy",Hayao Miyazaki,When two girls move to the country to be near ...,Japanese,8.196011
The Wind Rises (2013),PG-13,"Animation, Biography, Drama, History, Romance,...",Hayao Miyazaki,"A look at the life of Jiro Horikoshi, the man ...","Japanese, German, Italian, French",7.786849
The Boy Who Saw the Wind (2000),NaN,"Animation, Adventure, Drama",Kazuki Ohmori,"A boy has a powerful gift, a war hungery gover...","Japanese, Japanese",5.895389


In [46]:
return_tfidf_rec('Spirited Away (2001)', 'weighted_rating', movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Howl's Moving Castle (2004),PG,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,When an unconfident young woman is cursed with...,Japanese,8.196626
My Neighbor Totoro (1988),G,"Animation, Family, Fantasy",Hayao Miyazaki,When two girls move to the country to be near ...,Japanese,8.196011
Castle in the Sky (1986),PG,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,A young boy and a girl with a magic crystal mu...,Japanese,8.092852
Nausicaä of the Valley of the Wind (1984),PG,"Animation, Adventure, Fantasy, Sci-Fi",Hayao Miyazaki,Warrior and pacifist Princess Nausicaä despera...,"Japanese, English",8.092846
Porco Rosso (1992),PG,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"In 1930s Italy, a veteran World War I pilot is...",Japanese,7.786986
The Wind Rises (2013),PG-13,"Animation, Biography, Drama, History, Romance,...",Hayao Miyazaki,"A look at the life of Jiro Horikoshi, the man ...","Japanese, German, Italian, French",7.786849
Ponyo (2008),G,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,A five-year-old boy develops a relationship wi...,Japanese,7.692918
Mei and the Kitten Bus (2002),NaN,"Animation, Short, Comedy",Hayao Miyazaki,This is the story of Mei and her friend the ki...,Japanese,7.492856
Pom Poko (1994),PG,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,A community of magical shape-shifting raccoon ...,"Japanese, English",7.367225


In [30]:
return_tfidf_rec('Zootopia (2016)', 'weighted_rating', movies)

Here are the movie recommendations for Zootopia (2016): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Fantastic Mr. Fox (2009),PG,"Animation, Adventure, Comedy, Crime, Drama, Fa...",Wes Anderson,An urbane fox cannot resist returning to his f...,"English, French",7.895249
Tangled (2010),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Nathan Greno, Byron Howard",The magically long-haired Rapunzel has spent h...,English,7.797811
Porco Rosso (1992),PG,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"In 1930s Italy, a veteran World War I pilot is...",Japanese,7.786986
Wreck-It Ralph (2012),PG,"Animation, Adventure, Comedy, Family, Fantasy",Rich Moore,A video game villain wants to be a hero and se...,English,7.697783
Tangled Ever After (2012),G,"Animation, Short, Action, Comedy, Family, Musical","Nathan Greno, Byron Howard","Rapunzel and Flynn Rider have their wedding, b...",English,7.650855
Life in a Day (2011),PG-13,"Documentary, Drama","Loressa Clisby, Kevin Macdonald, Hiroaki Aikaw...",A documentary shot by film-makers all over the...,"English, Italian, Japanese, German, Spanish, I...",7.648430
A Matter of Loaf and Death (2008),NaN,"Animation, Short, Comedy, Crime, Family, Mystery",Nick Park,"Wallace and his dog, Gromit, open a bakery and...",English,7.546831
Bolt (2008),PG,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams",The canine star of a fictional sci-fi/action s...,English,6.897374
James and the Giant Peach (1996),PG,"Animation, Adventure, Family, Fantasy, Musical",Henry Selick,"An orphan, who lives with his two cruel aunts,...",English,6.693261


In [49]:
return_tfidf_rec('Zootopia (2016)', None, movies)

Here are the movie recommendations for Zootopia (2016): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Tangled Ever After (2012),G,"Animation, Short, Action, Comedy, Family, Musical","Nathan Greno, Byron Howard","Rapunzel and Flynn Rider have their wedding, b...",English,7.650855
The Secret Life of Pets (2016.0),PG,"Animation, Adventure, Comedy, Family","Chris Renaud, Yarrow Cheney(co-director)",The quiet life of a terrier named Max is upend...,English,6.498116
Wreck-It Ralph (2012),PG,"Animation, Adventure, Comedy, Family, Fantasy",Rich Moore,A video game villain wants to be a hero and se...,English,7.697783
Bolt (2008),PG,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams",The canine star of a fictional sci-fi/action s...,English,6.897374
Tangled (2010),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Nathan Greno, Byron Howard",The magically long-haired Rapunzel has spent h...,English,7.797811
Moana (2016),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Ron Clements, John Musker, Don Hall(co-directo...","In Ancient Polynesia, when a terrible curse in...",English,7.596768
Who Framed Roger Rabbit (1988),PG,"Animation, Adventure, Comedy, Crime, Family, F...",Robert Zemeckis,A toon-hating detective is a cartoon rabbit's ...,English,7.695482
Life in a Day (2011),PG-13,"Documentary, Drama","Loressa Clisby, Kevin Macdonald, Hiroaki Aikaw...",A documentary shot by film-makers all over the...,"English, Italian, Japanese, German, Spanish, I...",7.648430
The Family Fang (2015),R,"Comedy, Drama, Mystery",Jason Bateman,A brother and sister return to their family ho...,English,6.079133


In [50]:
return_tfidf_rec('The Lord of the Rings: The Fellowship of the Ring (2001)', 'weighted_rating', movies)

Here are the movie recommendations for The Lord of the Rings: The Fellowship of the Ring (2001): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
The Lord of the Rings: The Return of the King (2003),PG-13,"Adventure, Drama, Fantasy",Peter Jackson,Gandalf and Aragorn lead the World of Men agai...,"English, Quenya, Old English, Sindarin",8.899177
The Lord of the Rings: The Two Towers (2002),PG,"Adventure, Drama, Fantasy",Peter Jackson,While Frodo and Sam edge closer to Mordor with...,"English, Sindarin, Old English",8.699152
Harry Potter and the Deathly Hallows: Part 2 (2011),PG-13,"Adventure, Drama, Fantasy, Mystery",David Yates,"Harry, Ron, and Hermione search for Voldemort'...",English,8.098670
The Hobbit: An Unexpected Journey (2012),PG-13,"Adventure, Family, Fantasy",Peter Jackson,"A reluctant Hobbit, Bilbo Baggins, sets out to...",English,7.898836
The Hobbit: The Desolation of Smaug (2013),PG-13,"Adventure, Fantasy",Peter Jackson,"The dwarves, along with Bilbo Baggins and Gand...",English,7.798583
The Hobbit: The Battle of the Five Armies (2014),PG-13,"Adventure, Fantasy",Peter Jackson,Bilbo and company are forced to engage in a wa...,English,7.398522
Forgotten Silver (1995),NaN,Comedy,"Costa Botes, Peter Jackson",The film deals with the career of a supposedly...,"English, Cantonese",7.382748
The Frighteners (1996),R,"Comedy, Fantasy, Horror",Peter Jackson,After a tragic car accident that kills his wif...,English,7.093092
The Long and Short of It (2003),NaN,"Short, Drama",Sean Astin,A workman gets some unexpected help from two p...,English,6.627800


In [51]:
return_tfidf_rec('The Lord of the Rings: The Fellowship of the Ring (2001)', None, movies)

Here are the movie recommendations for The Lord of the Rings: The Fellowship of the Ring (2001): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
The Lord of the Rings: The Return of the King (2003),PG-13,"Adventure, Drama, Fantasy",Peter Jackson,Gandalf and Aragorn lead the World of Men agai...,"English, Quenya, Old English, Sindarin",8.899177
The Lord of the Rings: The Two Towers (2002),PG,"Adventure, Drama, Fantasy",Peter Jackson,While Frodo and Sam edge closer to Mordor with...,"English, Sindarin, Old English",8.699152
The Lord of the Rings (1978),PG,"Animation, Adventure, Fantasy",Ralph Bakshi,The Fellowship of the Ring embark on a journey...,"English, Sindarin",6.193183
The Hobbit: An Unexpected Journey (2012),PG-13,"Adventure, Family, Fantasy",Peter Jackson,"A reluctant Hobbit, Bilbo Baggins, sets out to...",English,7.898836
The Long and Short of It (2003),NaN,"Short, Drama",Sean Astin,A workman gets some unexpected help from two p...,English,6.627800
The Hobbit: The Battle of the Five Armies (2014),PG-13,"Adventure, Fantasy",Peter Jackson,Bilbo and company are forced to engage in a wa...,English,7.398522
Forgotten Silver (1995),NaN,Comedy,"Costa Botes, Peter Jackson",The film deals with the career of a supposedly...,"English, Cantonese",7.382748
The Hobbit: The Desolation of Smaug (2013),PG-13,"Adventure, Fantasy",Peter Jackson,"The dwarves, along with Bilbo Baggins and Gand...",English,7.798583
The Frighteners (1996),R,"Comedy, Fantasy, Horror",Peter Jackson,After a tragic car accident that kills his wif...,English,7.093092


In [73]:
return_tfidf_rec('Iron Man (2008)', None, movies)

Here are the movie recommendations for Iron Man (2008): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Iron Man 2 (2010),PG-13,"Action, Adventure, Sci-Fi",Jon Favreau,With the world now aware of his identity as Ir...,"English, French, Russian",6.999241
Iron Man 3 (2013),PG-13,"Action, Adventure, Sci-Fi",Shane Black,When Tony Stark's world is torn apart by a for...,English,7.199172
Avengers: Age of Ultron (2015.0),PG-13,"Action, Adventure, Sci-Fi",Joss Whedon,When Tony Stark and Bruce Banner try to jump-s...,"English, Korean",7.299072
Aftermath: Population Zero (2008),NaN,"Documentary, Sci-Fi",NaN,Ever wondered what the world would be like wit...,English,6.547931
Zathura: A Space Adventure (2005),PG,"Action, Adventure, Comedy, Family, Fantasy, Sc...",Jon Favreau,Two young brothers are drawn into an intergala...,English,6.197735
Jumper (2008),PG-13,"Action, Adventure, Sci-Fi, Thriller",Doug Liman,A teenager with teleportation abilities sudden...,"English, Italian, Japanese, Mandarin",6.099441
Spider-Man (2002),PG-13,"Action, Adventure, Sci-Fi",Sam Raimi,"When bitten by a genetically modified spider, ...",English,7.299045
Blackhawk: Fearless Champion of Freedom (1952),APPROVED,"Action, Adventure, Sci-Fi","Spencer Gordon Bennet, Fred F. Sears",Based on a successful comic book that began in...,English,5.923704
A Jihad for Love (2007),UNRATED,Documentary,Parvez Sharma,"A documentary on gay, lesbian, and transgender...","Arabic, Turkish, English, Persian, Urdu, Panja...",6.247858


## Analysis of TF-IDF Vectors and Cosine Similarity via dot product

We've found another way to predict and recommend movies. I actually really like what's recommended in this one, and think it's slightly better than the other one, with one caveat: in our first model, a movie rated 'G' would only return G or PG rated movies. However in this one, PG-13 or even R rated movies are appearing in searches for children's movies. 

One thing to consider this would be to tune the words included and include more copies of whatever rating I want to be important: including 3 G's might keep R or PG-13 rated movies from getting through, as other G or PG rated movies would be more similar to the vectors. I don't have as much of a problem with R rated movies having G rated recommendations, but it could be odd for them to show up.

# Weighting and Tuning TF-IDF 

In [8]:
movies = pd.read_csv('scraped-movies-with-wr-ty.csv')
movies = movies.drop_duplicates(subset='title_year',keep='first')
movies.shape

(167469, 20)

In [9]:
# Cosine Similarity Paring down - Want below 52,000 movies

threshold_votes = movies['imdb_votes'].quantile(.40)
quarter = movies['imdb_rating'].quantile(.30)

movies_sm = movies[(movies['imdb_votes'] >= threshold_votes) & (movies['imdb_rating'] >= quarter)
                    & (movies['year'] >= 1900) ]

movies_sm.shape

(54905, 20)

In [10]:
movies_sm = movies_sm[['title_year','rated','writer','genre','director','actors','plot','language','country', 'year']]

for column in movies_sm:
    movies_sm[column] = movies_sm[column].apply(lambda x: str(x))
    
# discarding the commas between the actors' full names
movies_sm['actors'] = movies_sm['actors'].map(lambda x: x.split(','))

# putting the genres in a list of words
movies_sm['genre'] = movies_sm['genre'].map(lambda x: x.lower().split(','))

movies_sm['director'] = movies_sm['director'].map(lambda x: x.split(','))


# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in movies_sm.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = [x.lower().replace(' ','') for x in row['director']]


# Getting rid of information between () in writers
movies_sm['writer'] = movies_sm['writer'].str.replace(r"\(.*\)","")

# discarding the commas between writer's names
movies_sm['writer'] = movies_sm['writer'].map(lambda x: x.split(','))

# putting the countries in a list of words
movies_sm['country'] = movies_sm['country'].map(lambda x: x.lower().split(','))

# putting the languages in a list of words
movies_sm['language'] = movies_sm['language'].map(lambda x: x.lower().split(','))

movies_sm['rated'] = movies_sm['rated'].str.replace("-","")
movies_sm['rated'] = movies_sm['rated'].apply(lambda x: str(x))
movies_sm['rated'] = movies_sm['rated'].apply(lambda x: [x,x] if x == 'G' or x == 'PG' else x.split(','))

movies_sm['year'] = movies_sm['year'].map(lambda x: x.split(','))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in movies_sm.iterrows():
    row['writer'] = [x.lower().replace(' ','') for x in row['writer']]
    row['rated'] = [x.lower().replace(' ','') for x in row['rated']]

movies_sm.head()

,title_year,rated,writer,genre,director,actors,plot,language,country,year
0,Toy Story (1995),"[g, g]",[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",A cowboy doll is profoundly threatened and jea...,[english],[usa],[1995]
1,Jumanji (1995),"[pg, pg]",[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...",When two kids find and play a magical board ga...,"[english, french]",[usa],[1995]
2,Grumpier Old Men (1995),[pg13],"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...",John and Max resolve to save their beloved bai...,"[english, italian, german]",[usa],[1995]
3,Waiting to Exhale (1995),[r],[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...","Based on Terry McMillan's novel, this film fol...",[english],[usa],[1995]
4,Father of the Bride Part II (1995),"[pg, pg]",[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",George Banks must deal not only with the pregn...,[english],[usa],[1995]


In [136]:
#movies_sm.to_csv('testing_rating_weights.csv', index=False)

In [11]:
# initializing the new column
movies_sm['Key_words'] = ""

for index, row in movies_sm.iterrows():
    plot = row['plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
movies_sm.drop(columns = ['plot'], inplace = True)

movies_sm.drop_duplicates(subset='title_year',keep='first')
movies_sm.shape

(54905, 10)

In [12]:
movies_sm.set_index('title_year', inplace = True)

movies_sm['bag_of_words'] = ''
columns = movies_sm.columns
for index, row in movies_sm.iterrows():
    words = ''
    for col in columns:
        #if col == 'director':
        #    words = words.join(row[col]) + ' '
        #else:
        words = words + ' '.join(row[col])+ ' '

    row['bag_of_words'] = words
    
movies_sm.drop(columns = [col for col in movies_sm.columns if col!= 'bag_of_words'], inplace = True)

movies_sm['bag_of_words'] = movies_sm['bag_of_words'].str.replace('nan','')

movies_sm.head()

,bag_of_words
title_year,
Toy Story (1995),g g johnlasseter animation adventure comedy ...
Jumanji (1995),pg pg jonathanhensleigh adventure comedy fam...
Grumpier Old Men (1995),pg13 markstevenjohnson markstevenjohnson comed...
Waiting to Exhale (1995),r terrymcmillan comedy drama romance forestw...
Father of the Bride Part II (1995),pg pg alberthackett comedy family romance ch...


In [169]:
#movies_sm.to_csv('June18-weighted-mpaa-rating-bow.csv')

In [13]:
vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), stop_words='english')
tfmat_w = vectorizer.fit_transform(movies_sm['bag_of_words'])

In [14]:
start = time.time()

tfw_cosine_sim = linear_kernel(tfmat_w, tfmat_w)
tfmat_w_time = time.time() - start
#times.append(tfmat_w_time)
print("Runtime %0.2f" % (time.time() - start))

Runtime 386.64


In [15]:
tfw_cosine_sim

array([[1.00000000e+00, 4.93125047e-02, 2.19236186e-02, ...,
        5.73451510e-04, 1.00346919e-02, 1.03071600e-03],
       [4.93125047e-02, 1.00000000e+00, 1.83736900e-02, ...,
        5.64895569e-04, 9.88497349e-03, 3.42182464e-03],
       [2.19236186e-02, 1.83736900e-02, 1.00000000e+00, ...,
        7.93773234e-03, 1.03937123e-03, 2.38187530e-03],
       ...,
       [5.73451510e-04, 5.64895569e-04, 7.93773234e-03, ...,
        1.00000000e+00, 6.02495857e-03, 4.14419226e-03],
       [1.00346919e-02, 9.88497349e-03, 1.03937123e-03, ...,
        6.02495857e-03, 1.00000000e+00, 9.68230247e-04],
       [1.03071600e-03, 3.42182464e-03, 2.38187530e-03, ...,
        4.14419226e-03, 9.68230247e-04, 1.00000000e+00]])

In [16]:
movies = pd.read_csv('/Users/user/final-capstone/scraped-movies-with-wr-ty.csv')

cols = ['title_year','rated','genre','director','plot','language','weighted_rating']
movies = movies[cols]
movies = movies.drop_duplicates(subset='title_year',keep='first')
movies = movies.set_index('title_year')

In [17]:
movies_sm = pd.read_csv('June18-weighted-mpaa-rating-bow.csv')

In [20]:
# Build a 1-dimensional array with movie titles
titles = movies_sm['title_year']
indices = pd.Series(movies_sm.index, index=movies_sm['title_year'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def tfidf_recommendations(title, cosine_sim=tfw_cosine_sim):
    
    '''
    This function returns a list of movies with the highest similarity scores other than itself
    '''
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


def return_tfidf_rec(title, orderby, df):
    
    '''
    using the tfidf_recommendations function, filters and sorts the recommendations by a column. Can order results
    by weighted rating or any other column value, if None then results are returned in order by cosine similarity
    '''
    
    title_a = tfidf_recommendations(title).tolist()
    title_recs = df.loc[title_a].reset_index(inplace=False)
    title_recs = title_recs.set_index('title_year')
    print("Here are the movie recommendations for {}: ".format(title))
    if orderby == None:
        return title_recs[:10]
    else:
        return title_recs[:10].sort_values(by=[orderby], ascending=False)

In [21]:
return_tfidf_rec('Spirited Away (2001)', None, movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Nausicaä of the Valley of the Wind (1984),PG,"Animation, Adventure, Fantasy, Sci-Fi",Hayao Miyazaki,Warrior and pacifist Princess Nausicaä despera...,"Japanese, English",8.092846
Porco Rosso (1992),PG,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"In 1930s Italy, a veteran World War I pilot is...",Japanese,7.786986
Castle in the Sky (1986),PG,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,A young boy and a girl with a magic crystal mu...,Japanese,8.092852
Howl's Moving Castle (2004),PG,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,When an unconfident young woman is cursed with...,Japanese,8.196626
Mei and the Kitten Bus (2002),NaN,"Animation, Short, Comedy",Hayao Miyazaki,This is the story of Mei and her friend the ki...,Japanese,7.492856
My Neighbor Totoro (1988),G,"Animation, Family, Fantasy",Hayao Miyazaki,When two girls move to the country to be near ...,Japanese,8.196011
Ponyo (2008),G,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,A five-year-old boy develops a relationship wi...,Japanese,7.692918
Pom Poko (1994),PG,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,A community of magical shape-shifting raccoon ...,"Japanese, English",7.367225
The Wind Rises (2013),PG-13,"Animation, Biography, Drama, History, Romance,...",Hayao Miyazaki,"A look at the life of Jiro Horikoshi, the man ...","Japanese, German, Italian, French",7.786849


In [22]:
return_tfidf_rec('Zootopia (2016)', None, movies)

Here are the movie recommendations for Zootopia (2016): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
The Secret Life of Pets (2016.0),PG,"Animation, Adventure, Comedy, Family","Chris Renaud, Yarrow Cheney(co-director)",The quiet life of a terrier named Max is upend...,English,6.498116
Wreck-It Ralph (2012),PG,"Animation, Adventure, Comedy, Family, Fantasy",Rich Moore,A video game villain wants to be a hero and se...,English,7.697783
Tangled Ever After (2012),G,"Animation, Short, Action, Comedy, Family, Musical","Nathan Greno, Byron Howard","Rapunzel and Flynn Rider have their wedding, b...",English,7.650855
Bolt (2008),PG,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams",The canine star of a fictional sci-fi/action s...,English,6.897374
Tangled (2010),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Nathan Greno, Byron Howard",The magically long-haired Rapunzel has spent h...,English,7.797811
Moana (2016),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Ron Clements, John Musker, Don Hall(co-directo...","In Ancient Polynesia, when a terrible curse in...",English,7.596768
Who Framed Roger Rabbit (1988),PG,"Animation, Adventure, Comedy, Crime, Family, F...",Robert Zemeckis,A toon-hating detective is a cartoon rabbit's ...,English,7.695482
The Pink Panther (2006),PG,"Adventure, Comedy, Crime, Family, Mystery",Shawn Levy,Bumbling Inspector Clouseau must solve the mur...,English,5.600504
Fantastic Mr. Fox (2009),PG,"Animation, Adventure, Comedy, Crime, Drama, Fa...",Wes Anderson,An urbane fox cannot resist returning to his f...,"English, French",7.895249


In [23]:
return_tfidf_rec('Brave (2012)', None, movies)

Here are the movie recommendations for Brave (2012): 


,rated,genre,director,plot,language,weighted_rating
title_year,,,,,,
Nanny McPhee (2005),PG,"Comedy, Family, Fantasy",Kirk Jones,A governess uses magic to rein in the behavior...,English,6.594460
The Prince of Egypt (1998),PG,"Animation, Adventure, Drama, Family, Fantasy, ...","Brenda Chapman, Steve Hickner, Simon Wells",An Egyptian prince learns of his identity as a...,"English, Hebrew",7.095095
The Legend of Mor'du (2012),NaN,"Animation, Short, Family",Brian Larsen,The witch from Pixar's Brave (2012) uses magic...,English,6.056765
One Man Band (2005),G,"Animation, Short, Comedy, Family, Music","Mark Andrews, Andrew Jimenez",Two buskers engage in a petty feud over a chil...,English,7.742367
Goosebumps 2: Haunted Halloween (2018),PG,"Adventure, Comedy, Family, Fantasy, Horror",Ari Sandel,Two young friends find a magic book that bring...,English,5.603379
ParaNorman (2012),PG,"Animation, Adventure, Comedy, Family, Fantasy","Chris Butler, Sam Fell","A misunderstood boy takes on ghosts, zombies a...",English,6.994317
Scooby-Doo! The Mystery Begins (2009),PG,"Animation, Adventure, Comedy, Family, Fantasy,...",Brian Levant,The story of how Mystery Inc. was formed.,English,5.607285
Wreck-It Ralph (2012),PG,"Animation, Adventure, Comedy, Family, Fantasy",Rich Moore,A video game villain wants to be a hero and se...,English,7.697783
Moana (2016),PG,"Animation, Adventure, Comedy, Family, Fantasy,...","Ron Clements, John Musker, Don Hall(co-directo...","In Ancient Polynesia, when a terrible curse in...",English,7.596768


# Assessment 

We're no longer getting R rated movies when we search for G rated movies, which is good. We do get one PG-13 rated movie when we search for Spirited Away, which is rated PG, so I'm ok with that 

This shows how changing the weight of keywords can impact the recommender algorithm.